In [2]:
import src.utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
stats = src.utils.json_load(snakemake.input.stats[0])
multiple = False
if "cis" in stats or "trans" in stats:
    multiple = True

In [ ]:
if multiple:
    stats_cis = stats.cis
    stats_trans = stats.trans
    stats_cis = pd.DataFrame.from_dict(stats_cis)
    stats_trans = pd.DataFrame.from_dict(stats_trans)
else:
    test_NOE = pd.read_json(snakemake.input.stats[0])
    display(test_NOE)

In [ ]:
if multiple:
    if "stat" in stats_cis:
        statistics_cis = pd.DataFrame(columns=stats_cis["stat"].to_list())
        print(statistics_cis)
        stat_up_cis = pd.DataFrame(columns=stats_cis["stat"].to_list())
        stat_low_cis = pd.DataFrame(columns=stats_cis["stat"].to_list())
        statistics_trans = statistics_cis.copy()
        print(statistics_trans)
        stat_up_trans = stat_up_cis.copy()
        stat_low_trans = stat_low_cis.copy()
    else:
        statistics_trans = pd.DataFrame(columns=stats_trans["stat"].to_list())
        print(statistics_trans)
        stat_up_trans = pd.DataFrame(columns=stats_trans["stat"].to_list())
        stat_low_trans = pd.DataFrame(columns=stats_trans["stat"].to_list())
        statistics_cis = statistics_trans.copy()
        print(statistics_cis)
        stat_up_cis = stat_up_trans.copy()
        stat_low_cis = stat_low_trans.copy()

else:
    statistics = pd.DataFrame(
        columns=test_NOE["single"]["stat"].keys()
    )  # .to_list()
    statistics
    stat_up = pd.DataFrame(
        columns=test_NOE["single"]["stat"].keys()
    )  # .to_list()
    stat_low = pd.DataFrame(
        columns=test_NOE["single"]["stat"].keys()
    )  # .to_list()

In [ ]:
if multiple:
    for json in snakemake.input.stats:
        NOE_combined = src.utils.json_load(json)
        # cis
        if "cis" in NOE_combined:
            NOE_cis = NOE_combined.cis
            NOE_cis = pd.DataFrame.from_dict(NOE_cis)
            NOE_cis = NOE_cis.set_index("stat")
            statistics_cis = statistics_cis.append(
                NOE_cis["value"], ignore_index=True
            )
            stat_up_cis = stat_up_cis.append(NOE_cis["up"], ignore_index=True)
            stat_low_cis = stat_low_cis.append(
                NOE_cis["low"], ignore_index=True
            )
        else:
            statistics_cis = statistics_cis.append([None], ignore_index=True)
            stat_up_cis = stat_up_cis.append([None], ignore_index=True)
            stat_low_cis = stat_low_cis.append([None], ignore_index=True)
        # trans
        if "trans" in NOE_combined:
            NOE_trans = NOE_combined.trans
            NOE_trans = pd.DataFrame.from_dict(NOE_trans)
            NOE_trans = NOE_trans.set_index("stat")
            statistics_trans = statistics_trans.append(
                NOE_trans["value"], ignore_index=True
            )
            stat_up_trans = stat_up_trans.append(
                NOE_trans["up"], ignore_index=True
            )
            stat_low_trans = stat_low_trans.append(
                NOE_trans["low"], ignore_index=True
            )
        else:
            statistics_trans = statistics_trans.append(
                [None], ignore_index=True
            )
            stat_up_trans = stat_up_trans.append([None], ignore_index=True)
            stat_low_trans = stat_low_trans.append([None], ignore_index=True)
else:
    for json in snakemake.input.stats:
        NOE = pd.read_json(json)
        #         display(NOE)
        #         NOE = NOE.set_index('stat')
        statistics = statistics.append(
            NOE["single"]["value"], ignore_index=True
        )
        stat_up = stat_up.append(NOE["single"]["up"], ignore_index=True)
        stat_low = stat_low.append(NOE["single"]["low"], ignore_index=True)

In [ ]:
inputs = []
for w in snakemake.input.stats:
    part1 = w.split("/")
    part1 = part1[5:9]
    part2 = w.split("/")[9][0:5]
    part1.append(part2)
    inputs.append("/".join(part1))

if multiple:
    statistics_cis["name"] = inputs
    statistics_cis = statistics_cis.set_index("name")
    statistics_trans["name"] = inputs
    statistics_trans = statistics_trans.set_index("name")
    if 0 in statistics_cis:
        statistics_cis = statistics_cis.drop(columns=[0])
        stat_up_cis = stat_up_cis.drop(columns=[0])
        stat_low_cis = stat_low_cis.drop(columns=[0])
    if 0 in statistics_trans:
        statistics_trans = statistics_trans.drop(columns=[0])
        stat_up_trans = stat_up_trans.drop(columns=[0])
        stat_low_trans = stat_low_trans.drop(columns=[0])
else:
    statistics["name"] = inputs
    statistics = statistics.set_index("name")
    statistics = statistics.rename(columns=test_NOE["single"]["stat"])
    stat_up = stat_up.rename(columns=test_NOE["single"]["stat"])
    stat_low = stat_low.rename(columns=test_NOE["single"]["stat"])

In [ ]:
if multiple:
    stat_up_cis["name"] = inputs
    stat_up_cis = stat_up_cis.set_index("name")
    stat_low_cis["name"] = inputs
    stat_low_cis = stat_low_cis.set_index("name")

    stat_up_trans["name"] = inputs
    stat_up_trans = stat_up_trans.set_index("name")
    stat_low_trans["name"] = inputs
    stat_low_trans = stat_low_trans.set_index("name")
else:
    stat_up["name"] = inputs
    stat_up = stat_up.set_index("name")

    stat_low["name"] = inputs
    stat_low = stat_low.set_index("name")

In [ ]:
if multiple:
    print(statistics_cis)
    print(statistics_trans)
else:
    print(statistics)

In [ ]:
if multiple:
    statistics_cis = statistics_cis.sort_values("RMSD")
    statistics_cis = statistics_cis.drop(columns=["chisq"])
    statistics_trans = statistics_trans.sort_values("RMSD")
    statistics_trans = statistics_trans.drop(columns=["chisq"])
else:
    statistics = statistics.sort_values("RMSD")
    statistics = statistics.drop(columns=["chisq"])
    statistics

In [ ]:
if multiple:
    stat_up_cis = stat_up_cis.reindex(statistics_cis.index)
    stat_low_cis = stat_low_cis.reindex(statistics_cis.index)
    stat_up_cis = stat_up_cis.drop(columns=["chisq"])
    stat_low_cis = stat_low_cis.drop(columns=["chisq"])
    #
    stat_up_trans = stat_up_trans.reindex(statistics_trans.index)
    stat_low_trans = stat_low_trans.reindex(statistics_trans.index)
    stat_up_trans = stat_up_trans.drop(columns=["chisq"])
    stat_low_trans = stat_low_trans.drop(columns=["chisq"])
else:
    stat_up = stat_up.reindex(statistics.index)
    stat_low = stat_low.reindex(statistics.index)
    stat_up = stat_up.drop(columns=["chisq"])
    stat_low = stat_low.drop(columns=["chisq"])

    stat_up

In [ ]:
if multiple:
    statistics_cis = statistics_cis.fillna(0)
    statistics_trans = statistics_trans.fillna(0)
    stat_up_cis = stat_up_cis.fillna(0)
    stat_low_cis = stat_low_cis.fillna(0)
    stat_up_trans = stat_up_trans.fillna(0)
    stat_low_trans = stat_low_trans.fillna(0)

In [ ]:
if multiple:
    annotation_cis = (
        stat_low_cis.round(2).astype(str)
        + "<"
        + statistics_cis.round(2).astype(str)
        + "<"
        + stat_up_cis.round(2).astype(str)
    )
    annotation_trans = (
        stat_low_trans.round(2).astype(str)
        + "<"
        + statistics_trans.round(2).astype(str)
        + "<"
        + stat_up_trans.round(2).astype(str)
    )
    fig, axs = plt.subplots(2, 1)
    fig.set_size_inches(18.5, 10.5)
    sns.heatmap(statistics_cis, ax=axs[0], annot=annotation_cis, fmt="")
    sns.heatmap(statistics_trans, ax=axs[1], annot=annotation_trans, fmt="")
    fig.suptitle("top: cis, bottom: trans")
    fig.savefig(snakemake.output.heatmap)
else:
    annotation = (
        stat_low.round(2).astype(str)
        + "<"
        + statistics.round(2).astype(str)
        + "<"
        + stat_up.round(2).astype(str)
    )
    print(annotation)
    fig, ax = plt.subplots()
    fig.set_size_inches(18.5, 10.5)
    ax = sns.heatmap(statistics, annot=annotation, fmt="")
    ax.set_ylabel("")
    fig.savefig(snakemake.output.heatmap, bbox_inches="tight")